# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics import *
from nltk.metrics.distance import (
    edit_distance,
    edit_distance_align,
    binary_distance,
    jaccard_distance,
    masi_distance,
    interval_distance,
    custom_distance,
    presence,
    fractional_presence,
)
from nltk.stem import SnowballStemmer
import re
from razdel import sentenize
from razdel import tokenize
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [23]:
import difflib

# читаем слова из файла
with open('litw-win.txt', 'r') as f:
    words = [word.strip() for word in f]

def correct_sentence(sentence):
    corrected = []
    for word in sentence.split():
        if word not in words:
            # ищем ближайшее слово из списка
            closest = difflib.get_close_matches(word, words, n=1)
            if closest:
                corrected.append(closest[0])
            else:
                corrected.append(word)
        else:
            corrected.append(word)
    return ' '.join(corrected)

text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''
correct = correct_sentence(text)
print(f'Orig: {text}')
print(f'Corr: {correct}')


Orig: с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий
Corr: с 26 величайшим 42 усилием 4 выбравшись из 6 потока 1 убегающих 761 людей 5 кутузову со 3 свитой 1 уменьшившейся 65 вдвое 2 поехало на 108 звуки 9 выстрелов 1 прусских 30 орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [2]:
# #разбиение на слова
# ww = 'Считайте слова из файла и запишите их в список. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка. Считайте, что в слове есть опечатка, если данное слово не содержится в списке. '
# z = ww.split()
# print(z)

#snb_stemmer_ru = SnowballStemmer('russian')
#for i in z:
    #print(snb_stemmer_ru.stem(i))
    
morphhhh = pymorphy2.MorphAnalyzer()
# for i in z:
#     print(morph.parse(i))

p = morphhhh.parse('кошка')
p

AttributeError: module 'inspect' has no attribute 'getargspec'

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).